# Random Network Distillation 

In [1]:
# TODO fix notebook reloading
%load_ext autoreload
%autoreload 2

In [2]:
import os
import gymnasium as gym
from gym.wrappers import RecordVideo
from IPython.display import Video, display, clear_output
from tqdm import tqdm
import torch 
# torch default device
if  torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# device = torch.device("mps")
torch.set_default_device(device)
print(f"Using device: {device}")

from rnd_rl.runner.policy_runner import PPOConfig, PolicyRunner

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using device: mps


In [3]:
# @title Visualization code. Used later.

def visualize(agent):

    video_dir = "./videos"  # Directory to save videos
    os.makedirs(video_dir, exist_ok=True)

    # Create environment with proper render_mode
    env = gym.make("InvertedPendulum-v5", render_mode="rgb_array", reset_noise_scale=0.2)

    # Apply video recording wrapper
    env = RecordVideo(env, video_folder=video_dir, episode_trigger=lambda x: True)

    obs, _ = env.reset()


    for t in range(4096):
        actions, _ = agent.get_action(torch.Tensor(obs)[None, :].to(device))
        actions = actions.detach()
        obs, _, done, _, _ = env.step(actions.squeeze(0).cpu().numpy())

        if done:
            # self.writer.add_scalar("Duration", t, i)
            break

    env.close()

    # Display the latest video
    video_path = os.path.join(video_dir, sorted(os.listdir(video_dir))[-1])  # Get the latest video


    clear_output(wait=True)
    display(Video(video_path, embed=True))

In [4]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir runs

In [5]:
n_envs = 64
envs = gym.vector.SyncVectorEnv(
    [lambda: gym.make("InvertedPendulum-v5", reset_noise_scale=0.2) for _ in range(n_envs)]
    )

### PPO baseline

In [6]:
ppo_cfg = PPOConfig(
    use_rnd=False, 
    clip_params=0.2,
    init_noise_std=1.0, 
    safety_layer_enabled=True,
)

In [7]:
num_epochs = 250
policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_cfg, 
    num_mini_epochs=10, 
    device=device, 
    experiment_name="PPO_woSafetyLayer"
)
for epoch in tqdm(range(num_epochs)):
    policy_runner.rollout(epoch)
    policy_runner.update()

wandb: Currently logged in as: yxia33928 (yxia33928-georgia-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


100%|██████████| 250/250 [13:16<00:00,  3.18s/it]


In [8]:
visualize(policy_runner.alg)
print("PPO trained agent")

PPO trained agent


### PPO with RND

In [9]:
ppo_rnd_cfg = PPOConfig(
    use_rnd=True, 
    clip_params=0.2,
    init_noise_std=1.0, 
)

In [10]:
num_epochs = 250 
rnd_policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_rnd_cfg, 
    num_mini_epochs=10,
    device=device,
    experiment_name="PPO_RND"
)
for epoch in tqdm(range(num_epochs)):
    rnd_policy_runner.rollout(epoch)
    rnd_policy_runner.update()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Extrinsic Reward,▁▅▇█████████████████████████████████████
Reward,▁▃▄█████████████████████████████████████
Extrinsic Reward,1
Reward,1


100%|██████████| 250/250 [12:57<00:00,  3.11s/it]


In [11]:
visualize(rnd_policy_runner.alg)
print("RND PPO trained agent")

RND PPO trained agent


### Reward normalization only

In [12]:

ppo_rnd_reward_normalization_cfg = PPOConfig(
    use_rnd=True, 
    clip_params=0.2,
    init_noise_std=1.0, 
    reward_normalization = True
)


In [13]:
num_epochs = 250 
rnd_reward_norm_policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_rnd_reward_normalization_cfg, 
    num_mini_epochs=10,
    device=device, 
    experiment_name="PPO_RND_normalization"
)
for epoch in tqdm(range(num_epochs)):
    rnd_reward_norm_policy_runner.rollout(epoch)
    rnd_reward_norm_policy_runner.update()

Extrinsic Reward,▁███████████████████████████████████████
Reward,█▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Extrinsic Reward,1
Reward,1.05156


100%|██████████| 250/250 [35:30<00:00,  8.52s/it]   


In [14]:
visualize(rnd_reward_norm_policy_runner.alg)
print("RND PPO trained agent with reward normalization")

RND PPO trained agent with reward normalization


### Reward and observation normalization

In [15]:
ppo_rnd_all_normalization_cfg = PPOConfig(
    use_rnd=True, 
    clip_params=0.2,
    init_noise_std=1.0, 
    reward_normalization = True,
    obs_normalization = True
)

In [16]:
num_epochs = 250 
rnd_all_norm_policy_runner = PolicyRunner(
    envs=envs, 
    policy_cfg=ppo_rnd_all_normalization_cfg, 
    num_mini_epochs=10,
    device=device, 
    experiment_name="PPO_RND_normalization"
)
for epoch in tqdm(range(num_epochs)):
    rnd_all_norm_policy_runner.rollout(epoch)
    rnd_all_norm_policy_runner.update()

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Extrinsic Reward,▁▆██████████████████████████████████████
Reward,▁▆▇▇████████████████████████████████████
Extrinsic Reward,1
Reward,1.00215


100%|██████████| 250/250 [14:52<00:00,  3.57s/it]


wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [17]:
visualize(rnd_all_norm_policy_runner.alg)
print("RND PPO trained agent with observation normalization")

RND PPO trained agent with observation normalization
